In [1]:
import pandas as pd
from datetime import datetime, timedelta
from io import StringIO
import requests
import unittest
import os
import shutil
import pytz

### clean folder structure

In [2]:
if os.path.exists("db"):
    shutil.rmtree("db")

### start demandForecastManager

In [3]:
!docker-compose down
!docker build -t demandforecastmanager .
!docker-compose up -d

[+] Running 0/0
 ⠋ Container demandforecastmanager-dfm-1  Stopping                         0.1s 
[+] Running 0/1
 ⠙ Container demandforecastmanager-dfm-1  Stopping                         0.2s 
[+] Running 0/1
 ⠹ Container demandforecastmanager-dfm-1  Stopping                         0.3s 
[+] Running 0/1
 ⠸ Container demandforecastmanager-dfm-1  Stopping                         0.4s 
[+] Running 2/1
 ✔ Container demandforecastmanager-dfm-1  Removed                          0.4s 
 ✔ Network demandforecastmanager_default  Removed                          0.1s 
ERROR: open /Users/ronnyschneeberger/.docker/buildx/activity/desktop-linux: permission denied
[+] Running 1/0
 ✔ Network demandforecastmanager_default  Created                          0.0s 
 ⠋ Container demandforecastmanager-dfm-1  Starting                         0.1s 
[+] Running 1/2
 ✔ Network demandforecastmanager_default  Created                          0.0s 
 ⠙ Container demandforecastmanager-dfm-1  Starting               

### run unit tests

In [4]:
base_url = "http://0.0.0.0:5011/"
class testDemandForecastManager(unittest.TestCase):
    base_url = base_url
    def test_0_index(self):
        response = requests.get(url=self.base_url)
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.text, '{"message":"Index of demandForecastManager."}\n')

    def test_1_get_health(self):
        response = requests.get(url=self.base_url + "getHealth")
        self.assertEqual(response.status_code, 200)

    def test_3_get_ids(self):
        response = requests.get(url=self.base_url + "getIds")
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.text, '{"buildingIds":[]}\n')

    def test_4_add_ids(self):
        headers = {
            "buildingIdentifier": "Building_Test",
            "smartmeterIdentifiers": 'Smartmeter_3,Smartmeter_4'
        }
        response = requests.post(url=self.base_url + "addId", headers=headers)
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.text, '{"message":"Created Database entry for Building_Test"}\n')
    
    def test_5_get_ids(self):
        # check smartmeter ids
        response = requests.get(url=self.base_url + "getIds")
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.text, '{"buildingIds":["Building_Test"]}\n')
        response = requests.get(url=self.base_url + "getSmartmeterIds", headers={'buildingIdentifier':'Building_Test'})
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.text, '{"smartmeterIdentifiers":["Smartmeter_3","Smartmeter_4"]}\n')

    def test_6_update_ids(self):
        headers = {
            'buildingIdentifier': 'Building_Test',
            'smartmeterIdentifiers': 'Smartmeter_4,Smartmeter_5,Smartmeter_6'
        }
        response = requests.put(url=self.base_url + "updateId", headers=headers)
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.text, '{"message":"Updated Smartmeter Ids for Building_Test"}\n')

    def test_7_get_ids(self):
        # TODO CHECK SMARTMETER IDS
        response = requests.get(url=self.base_url + "getIds")
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.text, '{"buildingIds":["Building_Test"]}\n')
        response = requests.get(url=self.base_url + "getSmartmeterIds", headers={'buildingIdentifier':'Building_Test'})
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.text, '{"smartmeterIdentifiers":["Smartmeter_3","Smartmeter_4","Smartmeter_5","Smartmeter_6"]}\n')

    def test_8_delete_smartmeter_ids(self):
        headers = {
            'buildingIdentifier': "Building_Test",
            'smartmeterIdentifiers': 'Smartmeter_3,Smartmeter_4'
        }
        
        response = requests.delete(url=self.base_url + "deleteSmartmeterId", headers=headers)
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.text, '{"message":"Smartmeter Identifiers successfully removed from database."}\n')

    def test_91_get_ids(self):
        response = requests.get(url=self.base_url + "getSmartmeterIds", headers={'buildingIdentifier':'Building_Test'})
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.text, '{"smartmeterIdentifiers":["Smartmeter_5","Smartmeter_6"]}\n')

    def test_99_delete_building_id(self):
        headers = {
            'buildingIdentifier': 'Building_Test'
        }
        response = requests.delete(url=self.base_url + "deleteBuildingId", headers=headers)
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.text, '{"message":"Building identifier successfully removed from database."}\n')

In [5]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..........
----------------------------------------------------------------------
Ran 10 tests in 0.508s

OK


### add data to database

In [6]:
# load data
data = pd.read_feather("../data/ckw open data/0400ed9084770f70cb4447cac53e9c12.feather")
building_id = "Building_0400ed9084770f70cb4447cac53e9c12"
smart_meter_id = data.columns[1].split("_delta")[0]

# add building id
headers = {
    'buildingIdentifier': building_id,
    'smartmeterIdentifiers': smart_meter_id
}
response = requests.post(base_url + "addId", headers=headers)
assert response.status_code == 200

In [7]:
response.text

'{"message":"Created Database entry for Building_0400ed9084770f70cb4447cac53e9c12"}\n'

In [8]:
# upload data for building
response = requests.post(base_url + "postMeasurements", headers=headers, json=data.to_json(orient="split"))
print(response.text)

{"message":"Wrote measurements to database"}



In [9]:
# retrieve data
start = datetime.fromisoformat(data.timestamp[0])
start = datetime.isoformat(start)
end = datetime.fromisoformat(data.timestamp.iloc[-1])
end = datetime.isoformat(end)

headers = {
    'buildingIdentifier': building_id,
    'start': start,
    'end': end
}

response = requests.get(base_url + "getMeasurements", headers=headers)
response.status_code

200

In [10]:
print(pd.read_json(StringIO(response.json()['data']), orient='split'))

               timestamp  Smartmeter_3_delta_energy
0    2024-04-30 22:00:00                      0.289
1    2024-04-30 22:15:00                      0.123
2    2024-04-30 22:30:00                      0.203
3    2024-04-30 22:45:00                      0.193
4    2024-04-30 23:00:00                      0.136
...                  ...                        ...
2971 2024-05-31 20:45:00                      0.056
2972 2024-05-31 21:00:00                      0.044
2973 2024-05-31 21:15:00                      0.023
2974 2024-05-31 21:30:00                      0.020
2975 2024-05-31 21:45:00                      0.030

[2976 rows x 2 columns]


### update data

In [11]:
data.iloc[0, 1] = 0.21
data.iloc[2, 1] = 0.21
headers = {
    'buildingIdentifier': building_id,
    'smartmeterIdentifiers': smart_meter_id
}
response = requests.post(base_url + "postMeasurements", headers=headers, json=data.to_json(orient="split"))
response.text

'{"message":"Wrote measurements to database"}\n'

In [12]:
# retrieve data
start = datetime.fromisoformat(data.timestamp[0])
start = datetime.isoformat(start)
end = datetime.fromisoformat(data.timestamp.iloc[-1])
end = datetime.isoformat(end)

headers = {
    'buildingIdentifier': building_id,
    'start': start,
    'end': end
}

response = requests.get(base_url + "getMeasurements", headers=headers)

In [13]:
pd.read_json(StringIO(response.json()['data']), orient='split')

,timestamp,Smartmeter_3_delta_energy
0,2024-04-30 22:00:00,0.210
1,2024-04-30 22:15:00,0.123
2,2024-04-30 22:30:00,0.210
3,2024-04-30 22:45:00,0.193
4,2024-04-30 23:00:00,0.136
...,...,...
2971,2024-05-31 20:45:00,0.056
2972,2024-05-31 21:00:00,0.044
2973,2024-05-31 21:15:00,0.023
2974,2024-05-31 21:30:00,0.020


### add new values

In [14]:
# creating new data
start = datetime.fromisoformat(data.timestamp.iloc[-1]) + timedelta(minutes=15)
end = start + timedelta(hours=24)
timestamps = pd.date_range(start=start, end=end, freq="15min", tz=pytz.utc).values
timestamps = [pd.Timestamp(timestamp, tz=pytz.utc).to_pydatetime().isoformat() for timestamp in timestamps]
values = data['Smartmeter_3_delta_energy'].values[:len(timestamps)]

new_data = pd.DataFrame({
    'timestamp': timestamps,
    'Smartmeter_3_delta_energy': values
})

headers = {
    'buildingIdentifier': building_id,
    'smartmeterIdentifiers': smart_meter_id
} 

response = requests.post(base_url + "postMeasurements", headers=headers, json=new_data.to_json(orient="split"))
response.text

'{"message":"Wrote measurements to database"}\n'

In [15]:
# retrieve data
start = datetime.fromisoformat(data.timestamp.iloc[0])
start = datetime.isoformat(start)
end = datetime.fromisoformat(data.timestamp.iloc[-1])
end = datetime.isoformat(end)

headers = {
    'buildingIdentifier': building_id,
    'start': start,
    'end': end
}

response = requests.get(base_url + "getMeasurements", headers=headers)
dump = pd.read_json(StringIO(response.json()['data']), orient='split')

In [16]:
test = pd.read_feather("db/data/Building_0400ed9084770f70cb4447cac53e9c12.feather")

In [17]:
test.sort_values(by='timestamp', ascending=True)

,timestamp,Smartmeter_3_delta_energy
0,2024-04-30 22:00:00+00:00,0.210
1,2024-04-30 22:15:00+00:00,0.123
2,2024-04-30 22:30:00+00:00,0.210
3,2024-04-30 22:45:00+00:00,0.193
4,2024-04-30 23:00:00+00:00,0.136
...,...,...
3068,2024-06-01 21:00:00+00:00,0.025
3069,2024-06-01 21:15:00+00:00,0.043
3070,2024-06-01 21:30:00+00:00,0.020
3071,2024-06-01 21:45:00+00:00,0.026


### test min and max

In [26]:
data = pd.read_feather("../data/ckw open data/0400ed9084770f70cb4447cac53e9c12.feather")
data.min(), data.max()

(timestamp                    2024-04-30T22:00:00.000Z
 Smartmeter_3_delta_energy                       0.006
 dtype: object,
 timestamp                    2024-05-31T21:45:00.000Z
 Smartmeter_3_delta_energy                       1.502
 dtype: object)

In [24]:
response = requests.get(base_url + "getMinMax", headers={'buildingIdentifier': 'Building_0400ed9084770f70cb4447cac53e9c12'})
assert 200 == response.status_code
response.text

'{"max":{"Smartmeter_3":1.5019999742507935},"min":{"Smartmeter_3":0.006000000052154064}}\n'